In [1]:
from pandora_bayesopt.test_functions.robot_pushing.robot_pushing import robot_pushing_14d

pygame 2.1.0 (SDL 2.0.16, Python 3.9.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import torch
from torch import Tensor
import numpy as np
from botorch.utils.sampling import draw_sobol_samples
# from botorch.acquisition import ExpectedImprovement
# from pandora_bayesopt.acquisition.ei_puc import ExpectedImprovementWithCost
# from pandora_bayesopt.acquisition.gittins import GittinsIndex
# from pandora_bayesopt.acquisition.budgeted_multi_step_ei import BudgetedMultiStepLookaheadEI
from pandora_bayesopt.bayesianoptimizer import BayesianOptimizer

In [3]:
# use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set default tensor type to float64
torch.set_default_dtype(torch.float64)

In [4]:
seed = 42
torch.manual_seed(seed)

In [5]:
target_location = 10.0 * torch.rand([2]) - 5.0
target_location2 = 10.0 * torch.rand([2]) - 5.0

In [6]:
target_location, target_location2

(tensor([-4.4185, -4.3709]), tensor([-3.7641, -4.4742]))

In [7]:
init_location = torch.tensor([0, 2])
init_location2 = torch.tensor([0, -2])

In [9]:
def unnorm_X(X: torch.Tensor) -> torch.Tensor:
    X_unnorm = X.clone()
    # Check if the tensor is higher than 3-dimensional
    if X.dim() > 3:
        # Assuming the extra unwanted dimension is at position 1 (the second position)
        X_unnorm = X_unnorm.view(-1, X.size(-1))  # Remove the singleton dimension
    # Check the dimensionality of X and adjust accordingly
    if X.dim() == 3:
        # Remove the singleton dimension assuming it's the second one
        X_unnorm = X_unnorm.squeeze(1)
    elif X.dim() == 1:
        # If 1-dimensional, add a dimension to make it 2D (e.g., for batch size of 1)
        X_unnorm = X_unnorm.unsqueeze(0)
    X_unnorm[:, :2] = 10.0 * X_unnorm[:, :2] - 5.0
    X_unnorm[:, 2:4] = 5 * X_unnorm[:, 2:4]
    X_unnorm[:, 4] = 29.0 * X_unnorm[:, 4] + 1.0
    X_unnorm[:, 5] = 2 * np.pi * X_unnorm[:, 5]
    X_unnorm[:, 6:8] = 10.0 * X_unnorm[:, 6:8] - 5.0
    X_unnorm[:, 8:10] = 5 * X_unnorm[:, 8:10]
    X_unnorm[:, 10] = 29 * X_unnorm[:, 10] + 1.0
    X_unnorm[:, 11] = 2 * np.pi * X_unnorm[:, 11]
    X_unnorm[:, 12:] = 2 * np.pi * X_unnorm[:, 12:]

    return X_unnorm

def objective_cost_function(X: torch.Tensor) -> torch.Tensor:
    X_unnorm = unnorm_X(X)
    objective_X = []
    cost_X = []
    
    for x in X_unnorm:
        np.random.seed(0)
        object_location, object_location2, robot_location, robot_location2 = torch.tensor(robot_pushing_14d(x[0].item(), x[1].item(), x[2].item(), x[3].item(), x[4].item(), x[5].item(), x[6].item(), x[7].item(), x[8].item(), x[9].item(), x[10].item(), x[11].item(), x[12].item(), x[13].item()))
        objective_X.append(-torch.dist(target_location, object_location)-torch.dist(target_location2, object_location2))
        moving_distance = torch.dist(x[:2], robot_location)+torch.dist(x[6:8], robot_location2)+0.1
        
        cost_X.append(moving_distance)

    np.random.seed()  # Reset the seed
    
    objective_X = torch.tensor(objective_X)
    cost_X = torch.tensor(cost_X)
    return objective_X, cost_X

In [9]:
seed = 1
torch.manual_seed(seed)

In [10]:
dim = 14
bounds = torch.stack([torch.zeros(dim), torch.ones(dim)])
init_x = draw_sobol_samples(bounds=bounds, n=1, q=2*(dim+1)).squeeze(0)

In [11]:
objective_X, cost_X = objective_cost_function(init_x)

In [12]:
objective_X, cost_X

(tensor([-14.7659, -12.2576, -12.2576, -12.2576, -12.2576, -13.4437, -12.2576,
         -12.2576, -12.2576, -12.2576, -12.2576, -12.2576, -14.0308, -12.2576,
         -12.3664, -12.6729, -12.3685, -12.2576, -12.8875, -12.2576, -13.5099,
         -12.2576, -12.7769, -12.3122, -12.2576, -12.2576, -12.2576, -11.9910,
         -11.9794, -12.2576]),
 tensor([15.0645, 13.4399, 11.7955,  6.0913, 14.3644, 18.2455, 17.2731,  7.6569,
         15.0609,  5.5775,  4.8969, 17.2050, 15.4101, 14.1806,  8.8827,  9.4806,
         12.4590,  7.5306,  3.0377,  6.4146,  8.6321, 16.3050,  5.3135, 15.6288,
         17.6221,  6.5471, 16.5901, 16.6528, 11.4678,  8.5962]))

In [13]:
Optimizer = BayesianOptimizer(
        dim=dim, 
        maximize=True, 
        initial_points=init_x,
        objective_cost=objective_cost_function, 
        input_standardize=True
    )
Optimizer.run_until_budget(
    budget = 10,
    acquisition_function_class=BudgetedMultiStepLookaheadEI
)

357.42267252032934
Iteration 0, New point: [0.14135802 0.74886461 0.4257125  0.46840969 0.1964887  0.84170694
 0.34290749 0.67712155 0.35569603 0.61594825 0.25810555 0.34866355
 0.12162245 0.46747142], New value: -12.456405716506502
Best observed value: -11.979440263560832
Current acquisition value: 0.07769895428256035
Cumulative cost: 4.457966623134293
Running time: 1756.5769879999998

357.4226725203293


/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOPT/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/gpytorch.py:99: BotorchTensorDimensionWarning: Non-strict enforcement of botorch tensor conventions. The following error would have been raised with strict enforcement: Expected X and Y to have the same number of dimensions (got X with dimension 3 and Y with dimension 4).
  warnings.warn(
/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOPT/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/gpytorch.py:99: BotorchTensorDimensionWarning: Non-strict enforcement of botorch tensor conventions. The following error would have been raised with strict enforcement: Expected X and Y to have the same number of dimensions (got X with dimension 4 and Y with dimension 5).
  warnings.warn(
/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOPT/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/gpytorch.py:99: BotorchTens

Iteration 1, New point: [0.12851603 0.73105607 0.40261744 0.47537839 0.20244375 0.79228009
 0.33864227 0.64284207 0.31570243 0.6479134  0.21184494 0.3715364
 0.46730938 0.31453575], New value: -13.060730897545515
Best observed value: -11.979440263560832
Current acquisition value: 0.0349977671449512
Cumulative cost: 8.038841429721792
Running time: 277.72485099999994

357.4226725203293


/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOPT/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/gpytorch.py:99: BotorchTensorDimensionWarning: Non-strict enforcement of botorch tensor conventions. The following error would have been raised with strict enforcement: Expected X and Y to have the same number of dimensions (got X with dimension 3 and Y with dimension 4).
  warnings.warn(
/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOPT/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/gpytorch.py:99: BotorchTensorDimensionWarning: Non-strict enforcement of botorch tensor conventions. The following error would have been raised with strict enforcement: Expected X and Y to have the same number of dimensions (got X with dimension 4 and Y with dimension 5).
  warnings.warn(
/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOPT/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/gpytorch.py:99: BotorchTens

Iteration 2, New point: [0.26063612 0.58015562 0.36887043 0.46410504 0.28327847 0.60813688
 0.45259113 0.46869818 0.31936337 0.69175298 0.08704207 0.43504405
 0.77908965 0.74146004], New value: -12.441184528715397
Best observed value: -11.979440263560832
Current acquisition value: 0.00066877526108295
Cumulative cost: 11.439752058082547
Running time: 325.09786099999997



In [14]:
def objective_function(X: torch.Tensor) -> torch.Tensor:
        X_unnorm = unnorm_X(X)
        objective_X = []
        for x in X_unnorm:
                # Set the seed based on X to ensure consistent randomness
                np.random.seed(0)
                object_location, object_location2, robot_location, robot_location2 = torch.tensor(robot_pushing_14d(x[0].item(), x[1].item(), x[2].item(), x[3].item(), x[4].item(), x[5].item(), x[6].item(), x[7].item(), x[8].item(), x[9].item(), x[10].item(), x[11].item(), x[12].item(), x[13].item()))
                objective_X.append(-torch.dist(target_location, object_location)-torch.dist(target_location2, object_location2))
        np.random.seed()  # Reset the seed
        return torch.tensor(objective_X)
        
def cost_function(X: torch.Tensor) -> torch.Tensor:
        X_unnorm = unnorm_X(X)
        return torch.max(X_unnorm[:, 4], X_unnorm[:, 10]).unsqueeze(0)

In [16]:
seed = 1
torch.manual_seed(seed)
dim = 14
bounds = torch.stack([torch.zeros(dim), torch.ones(dim)])
init_x = draw_sobol_samples(bounds=bounds, n=1, q=2*(dim+1)).squeeze(0)

In [17]:
from botorch.acquisition.max_value_entropy_search import qMultiFidelityMaxValueEntropy
Optimizer = BayesianOptimizer(
    dim=dim, 
    maximize=True, 
    initial_points=init_x,
    objective=objective_function, 
    cost=cost_function,
    output_standardize=True
    )
Optimizer.run_until_budget(
    budget = 20,
    acquisition_function_class=qMultiFidelityMaxValueEntropy
)

Iteration 0, New point: [0.89050487 0.92320345 0.45557586 0.82986061 0.87352606 0.34166862
 0.32103348 0.56693015 0.02535154 0.57833602 0.69243414 0.21544393
 0.60248031 0.18759451], New value: -12.257621159724565
Best observed value: -11.979440263560832
Current acquisition value: -8.083271503692927e-08
Cumulative cost: 26.33225563261658
Running time: 12.712875

